In [1]:
import numpy as np
import tensorflow  as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from matplotlib import cm
from sklearn.manifold import TSNE
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Loading Dataset
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

#Number of hidden layers
n_hl = 5

#Number of neurons in each layer = 1024
n_nodes_hl = [1024]*n_hl

#size of vector of each sample
n_inputs = 28*28

#Number of output classes
n_class = 10

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
#Declaring X, y placefolder for input and output
X = tf.placeholder(tf.float32, [None, n_inputs], name="X") 
y = tf.placeholder(tf.int64, [None,10], name="y")

In [4]:
#Neural Network with specified hidden layers 
with tf.name_scope("dnn"):
  
  hidden_layer = [0 for i in range(n_hl)]
  
  hidden_layer[0] = tf.layers.dense(X, n_nodes_hl[0], tf.nn.relu, name='hl1')
    
  for i in range(1,n_hl):
    hl_name = 'hl'+str(i+1)
    print(hl_name)
    hidden_layer[i] = tf.layers.dense(hidden_layer[i-1],n_nodes_hl[i], tf.nn.relu, name=hl_name)
  
  last_hl = hidden_layer[-1]
  
  logits = tf.layers.dense(hidden_layer[-1], n_class, activation=None, name='output')
  logits_sm = tf.nn.softmax(logits)  #softmax output

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
hl2
hl3
hl4
hl5


In [5]:
print(logits)

Tensor("dnn/output/BiasAdd:0", shape=(?, 10), dtype=float32)


In [0]:
#Cross entropy
with tf.name_scope("loss"):
  xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
  loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
learning_rate = 0.001

with tf.name_scope("train"):
  optimizer = tf.train.AdamOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)


In [0]:
with tf.name_scope("eval"):
  correct = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [10]:
#Training the NN
n_epochs = 101
batch_size = 50
weights = [0 for i in range(n_hl+1)]
biases = [0 for i in range(n_hl+1)]

with tf.Session() as sess:
  init.run()
  for epoch in range(n_epochs):
    for iteration in range(mnist.train.num_examples // batch_size):
      X_batch, y_batch = mnist.train.next_batch(batch_size)
      #print(X_batch.shape)
      sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y:mnist.test.labels})
    if epoch % 10 == 0:
      print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
      
  save_path = saver.save(sess, "./mnist_model.ckpt")

0 Train accuracy: 1.0 Test accuracy: 0.966
10 Train accuracy: 1.0 Test accuracy: 0.9777
20 Train accuracy: 0.98 Test accuracy: 0.9758
30 Train accuracy: 1.0 Test accuracy: 0.9841
40 Train accuracy: 1.0 Test accuracy: 0.982
50 Train accuracy: 1.0 Test accuracy: 0.9839
60 Train accuracy: 1.0 Test accuracy: 0.9836
70 Train accuracy: 0.98 Test accuracy: 0.9815
80 Train accuracy: 1.0 Test accuracy: 0.9816
90 Train accuracy: 1.0 Test accuracy: 0.9818
100 Train accuracy: 1.0 Test accuracy: 0.9854


In [11]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'hl1/kernel:0' shape=(784, 1024) dtype=float32_ref>,
 <tf.Variable 'hl1/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'hl2/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'hl2/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'hl3/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'hl3/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'hl4/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'hl4/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'hl5/kernel:0' shape=(1024, 1024) dtype=float32_ref>,
 <tf.Variable 'hl5/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'output/kernel:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'output/bias:0' shape=(10,) dtype=float32_ref>]

In [12]:
#Outputting the weights and biases
with tf.Session() as sess:
  saver.restore(sess, "./mnist_model.ckpt")
  weights[0] = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'hl1/kernel:0'))
  for l in range(1,n_hl):
    layer_name = 'hl'+str(l+1)+'/kernel:0'
    #print(layer_name)
    weights[l] = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, layer_name))
  weights[5] = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'output/kernel:0'))

  biases[0] = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'hl1/bias:0'))
  for l in range(1,n_hl):
    layer_name = 'hl'+str(l+1)+'/bias:0'
    #print(layer_name)
    biases[l] = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, layer_name))
  biases[5] = sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'output/bias:0'))

INFO:tensorflow:Restoring parameters from ./mnist_model.ckpt


In [13]:
weights[0]

[array([[ 0.02256305, -0.04756363,  0.01568658, ...,  0.03672467,
          0.04900185,  0.0196046 ],
        [ 0.00972131,  0.03792369,  0.04423331, ..., -0.00421304,
         -0.04831158, -0.02925314],
        [ 0.04968909,  0.04003197, -0.03157269, ..., -0.02185892,
          0.03120586,  0.00199647],
        ...,
        [-0.04404537,  0.00144889,  0.00549342, ..., -0.0272951 ,
          0.00364791, -0.02948521],
        [-0.04351641,  0.02487745,  0.00029998, ...,  0.04417834,
          0.04857656, -0.02395961],
        [ 0.03215345, -0.02691865, -0.03697591, ..., -0.01438579,
          0.05096023,  0.04501759]], dtype=float32)]

In [14]:
biases[0]

[array([-0.15123259, -0.16778518, -0.07526848, ..., -0.05836593,
         0.2359232 , -0.01319878], dtype=float32)]

In [15]:
#Checking accuracy for extracted weights
def relu(x):
  return np.maximum(x,0)

layer0 = relu(mnist.test.images@weights[0]+biases[0])
layer1 = relu(layer0@weights[1]+biases[1])
layer2 = relu(layer1@weights[2]+biases[2])
layer3 = relu(layer2@weights[3]+biases[3])
layer4 = relu(layer3@weights[4]+biases[4])
output = layer4@weights[5]+biases[5]

correct = np.equal(np.argmax(output[0],1), np.argmax(mnist.test.labels,1))
accuracy = np.mean(correct)
  
print('Accuracy:', accuracy)
  


Accuracy: 0.9854


In [0]:
#Singular value decomposition
S = [0]*(n_hl)  #tensor of singular values
U = [0]*(n_hl)  #tensor of left singular vectors
Vt = [0]*(n_hl)  #tensor of right singular vectors

for l in range(0,n_hl):
  U[l], S[l], Vt[l] = np.linalg.svd(weights[l], full_matrices=True)


In [0]:
#Defining new weights
new_weights = {}
d_lengths = [10,20,50,100,200,784]
for d in d_lengths:
  name = 'svd'+str(d)
  #new_weights[name] = [tf.matmul(U[l][0][:,0:d], tf.matmul(tf.linalg.diag(S[l][0][:d]),tf.transpose(V[l][0])[:d,:])) for l in range(0,n_hl)]
  new_weights[name] = [np.dot(U[l][0][:,:d], np.dot(np.diag(S[l][0][:d]), Vt[l][0][:d,:])) for l in range(0,n_hl)]
  new_weights[name].append(weights[-1][0])
  

In [0]:
#Compressed NN

def svd_nn_model(d):
  layer = [0 for i in range(n_hl)]
  #print(layer)
  name = 'svd'+str(d)
  layer[0] = mnist.test.images@new_weights[name][0]+biases[0]
  layer[0] = relu(layer[0])
  
  for i in range(1,len(n_nodes_hl)):
    layer[i] = layer[i-1]@new_weights[name][i]+biases[i]
    layer[i] = relu(layer[i])
    
  logits = layer[-1]@new_weights[name][-1]+biases[-1]
  #logits = tf.nn.softmax
  
  correct = np.equal(np.argmax(logits,1), np.argmax(mnist.test.labels,1))
  accuracy = np.mean(correct)

  
  return accuracy

In [19]:
svd_nn_model(784)

0.9854

In [20]:
#Accuracies for compressed NN
for d in d_lengths:
  print('Number of singular matrix elements:', d, ',Accuracy:',svd_nn_model(d))

Number of singular matrix elements: 10 ,Accuracy: 0.1018
Number of singular matrix elements: 20 ,Accuracy: 0.169
Number of singular matrix elements: 50 ,Accuracy: 0.8645
Number of singular matrix elements: 100 ,Accuracy: 0.9801
Number of singular matrix elements: 200 ,Accuracy: 0.9847
Number of singular matrix elements: 784 ,Accuracy: 0.9854


### 1.6 A

In [0]:
#Defining s,u,vt with 20 singular values
u = [U[l][0][:,0:20] for l in range(n_hl)]
s = [np.diag(S[l][0][:20]) for l in range(n_hl)]
vt = [Vt[l][0][0:20,:] for l in range(n_hl)]

In [0]:
v_bar = [s[l]@vt[l] for l in range(n_hl)]

In [23]:
#Initial U, V_bar matrix
#print('U',u[5][0:5])
print('V_bar',v_bar[4][0])

V_bar [-0.43047875 -0.4459179  -0.7832403  ... -0.75471675 -0.5920078
 -0.5116921 ]


In [0]:
U_new = [tf.Variable(tf.convert_to_tensor(u[i])) for i in range(n_hl)]
V_bar_new = [tf.Variable(tf.convert_to_tensor(v_bar[i])) for i in range(n_hl)]

In [0]:
def neural_network_model_w_svd_vars(data):
  global U_new
  global V_bar_new
  
  #Reconstructing the weights
  weights_updated = [tf.matmul(U_new[i],V_bar_new[i]) for i in range(n_hl)]
  
  #Declaring weights and biases for each layer
  hidden_layer = [0 for i in range(len(n_nodes_hl))]
  hidden_layer[0] = {'weights':weights_updated[0],
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl[0]]))}
  
  for i in range(1,len(n_nodes_hl)):
      hidden_layer[i]={'weights':weights_updated[i],
                       'biases':tf.random_normal([n_nodes_hl[i]])}
      
  output_layer = {'weights':weights[-1][0],
                   'biases':tf.Variable(tf.random_normal([n_class]))}
  
  layer = [0 for i in range(len(n_nodes_hl))]
  layer[0] = tf.add(tf.matmul(data, hidden_layer[0]['weights']), hidden_layer[0]['biases'])
  layer[0] = tf.nn.relu(layer[0])
  
  for i in range(1,len(n_nodes_hl)):
    layer[i] = tf.add(tf.matmul(layer[i-1], hidden_layer[i]['weights']), hidden_layer[i]['biases'])
    layer[i] = tf.nn.relu(layer[i])
    
  output = tf.matmul(layer[-1], output_layer['weights']) + output_layer['biases']
  
  return output, U_new, V_bar_new

In [26]:
#Training new model with U and V_bar
prediction, u, v = neural_network_model_w_svd_vars(X)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)

correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct,'float'))

hm_epochs = 101

with tf.Session() as sess:
  sess.run(tf.initialize_all_variables())

  for epoch in range(hm_epochs): 
    epoch_loss = 0
    for _ in range(int(mnist.train.num_examples/batch_size)):
      epoch_x, epoch_y = mnist.train.next_batch(batch_size)
      _, c = sess.run([optimizer, cost], feed_dict = {X: epoch_x, y: epoch_y})
      epoch_loss += c
    if epoch % 10 ==0:
      print('Epoch',epoch, epoch_loss)
      print('Train Accuracy:', accuracy.eval({X:epoch_x, y:epoch_y}))
      print('Updated final set of V_bar',sess.run(v[4][0]))


  print('Test Accuracy:', accuracy.eval({X:mnist.test.images, y:mnist.test.labels}))

  save_path = saver.save(sess, "./mnist_test_svd.ckpt")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 526.1912607548293
Train Accuracy: 0.94
Updated final set of V_bar [-0.42805848 -0.4456998  -0.7839892  ... -0.7545684  -0.59197265
 -0.5156501 ]
Epoch 10 38.22765079068108
Train Accuracy: 1.0
Updated final set of V_bar [-0.44149658 -0.4623381  -0.7776793  ... -0.76162463 -0.60076696
 -0.5306485 ]
Epoch 20 16.85299705095455
Train Accuracy: 1.0
Updated final set of V_bar [-0.440174   -0.4632443  -0.7869192  ... -0.7697097  -0.59643143
 -0.5361204 ]
Epoch 30 12.3217101872388
Train Accuracy: 0.98
Updated final set of V_bar [-0.43578255 -0.46919587 -0.79562014 ... -0.7599847  -0.58230764
 -0.5295787 ]
Epoch 40 10.796517194537714
Train Accuracy: 1.0
Updated final set of V_bar [-0.41113234 -0.4612933  -0.7985977  ... -0.6979